# Установка и импорт библиотек

In [1]:
!pip install -q SALib

In [2]:
import time

import numpy as np
from numba import njit
from SALib.analyze import sobol
from SALib.sample import saltelli

# 1 Анализ чувствительности, проверка сходимости, измерение таймингов

In [3]:
def evaluate_model(x: np.ndarray):
    result = x[0] ** 5 \
             - x[0]**4 * x[1] \
             + x[0]**3 * x[1]**2 * x[2] \
             - x[0]**2 * x[1]**3 * x[2]**2 * x[3] \
             + x[0]    * x[1]**4 * x[2]**3 * x[3]**2 * x[4]
    return result

In [4]:
NUM_ITERATIONS = 2 ** 17
NUM_VARIABLES = 5

In [5]:
problem = {
    "num_vars": NUM_VARIABLES,
    "names": [f"x{i + 1}" for i in range(NUM_VARIABLES)],
    "bounds": [[-1.0, 1.0] for _ in range(NUM_VARIABLES)]
}

In [6]:
start_time = time.time()

param_values = saltelli.sample(problem, NUM_ITERATIONS)

print(f"Samples generation took {time.time() - start_time:.6f} seconds")
print(f"param_values shape: {param_values.shape}")

Samples generation took 14.154654 seconds
param_values shape: (1572864, 5)


In [7]:
y = np.zeros([param_values.shape[0]])

In [8]:
start_time = time.time()

for i, x in enumerate(param_values):
    y[i] = evaluate_model(x)

eval_time = time.time() - start_time

print(f"Model evaluation took {eval_time:.6f} seconds")

Model evaluation took 13.352504 seconds


In [9]:
start_time = time.time()

sobol_indices = sobol.analyze(problem, y)

print(f"Sobol analysis took {time.time() - start_time:.6f} seconds")

Sobol analysis took 9.898124 seconds


In [10]:
def print_sobol_analysis_results(results, num_variables: int) -> None:
    print("First-order indices:")
    print(results["S1"])

    print("\nSecond-order indices:")
    for i in range(num_variables):
        for j in range(i + 1, num_variables):
            print(f"x{i + 1}-x{j + 1}: {results['S2'][i, j]}")
    
    print("\nTotal indices for each variable:")
    print(results["ST"])

In [11]:
print_sobol_analysis_results(sobol_indices, NUM_VARIABLES)

First-order indices:
[6.50612740e-01 9.58546455e-02 2.04748855e-04 2.46264223e-05
 1.06867457e-04]

Second-order indices:
x1-x2: 0.16987438126484783
x1-x3: 0.03826275403003994
x1-x4: 0.00020946561717304194
x1-x5: 0.0004444833633367216
x2-x3: -0.00011958479017451536
x2-x4: 0.004237998244049233
x2-x5: -7.914446571177364e-05
x3-x4: -4.5972631285220394e-05
x3-x5: 0.0002582995960626094
x4-x5: 0.0004137380424572515

Total indices for each variable:
[0.89695428 0.31024338 0.07663299 0.01500546 0.00255642]


# 2 Ускорение с помощью `numba`

In [12]:
@njit
def evaluate_model(x: np.ndarray):
    result = x[0]**5 \
             - x[0]**4 * x[1] \
             + x[0]**3 * x[1]**2 * x[2] \
             - x[0]**2 * x[1]**3 * x[2]**2 * x[3] \
             + x[0]    * x[1]**4 * x[2]**3 * x[3]**2 * x[4]
    return result

In [13]:
start_time = time.time()

for i, x in enumerate(param_values):
    y[i] = evaluate_model(x)

eval_time_numba = time.time() - start_time

print(f"Model evaluation took {eval_time_numba:.6f} seconds")

Model evaluation took 1.450562 seconds


In [14]:
print(f"Using numba resulted in acceleration by "
      f"{eval_time / eval_time_numba:.2f} times.")

Using numba resulted in acceleration by 9.21 times.
